In [ ]:
import json
import os

import sys
sys.path.append('..')
sys.path.append('../ark')

# NOTE: just for visualizing on blank space for the time being
import numpy as np

from ark.mibi import tiling_utils

# 1. Automatically generate tiled regions

This automatically creates tiles which define FOVs on the MIBI slide based on user-set parameters.

### Define paths to the JSON data

In [ ]:
base_dir = "../data/example_dataset"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")
fov_list_path = os.path.join(json_tiling_dir, 'fov_list_single_fov_tma.json')
moly_path = os.path.join(json_tiling_dir, 'moly_point.json')
tiled_output_path = os.path.join(json_tiling_dir, 'tiled_output_tma.json')

### Define whether TMA is used or not

In [ ]:
# tma = False
tma = True

### Set tiling parameters

In [ ]:
# list of parameters:
# starting x coordinate and y coordinate for each fov (taken from fov_list_file)
# number of fovs along the x- and y-axis for each fov (set by user)
# x- and y-axis step size (set by user)
# randomize the order of the fovs or not (set by user)
tiling_params, moly_point = tiling_utils.set_tiling_params(fov_list_path, moly_path, tma=tma)

### Create the FOV tiles for each region

In [ ]:
tiled_regions = tiling_utils.create_tiled_regions(tiling_params, moly_point, tma=tma)

In [ ]:
# write the tiled output
with open(tiled_output_path, 'w') as top:
    json.dump(tiled_regions, top)

# 2. Slide overlay QC (for TMAs only)

Using a pre-defined JSON of FOVs, compares with the automatically-generated FOVs from the previous step and allows for remapping if necessary.

### Define QC parameters, read data in

NOTE: the data contained in `proposed_tiled_output_path` is assumed to have the same tile dimensions and the same Moly point as the data in `tiled_output_path`.

In [ ]:
proposed_tiled_output_path = os.path.join(json_tiling_dir, 'tiled_output_tma_proposed.json')
slide_path = os.path.join(json_tiling_dir, 'slide.jpg')

In [ ]:
with open(proposed_tiled_output_path, 'r') as ptop:
    tiled_regions_proposed = json.load(ptop)

In [ ]:
# NOTE: for time being, just visualize on a blank space, this will be changed to the actual slide with co-registration
slide_data = np.full((1000, 1000, 3), 255)

### Map proposed tiles to their closest automatically-generated tile (Euclidean)

In [ ]:
proposed_to_auto_map, proposed_tiles_info, auto_tiles_info = tiling_utils.assign_closest_tiled_regions(tiled_regions_proposed, tiled_regions, moly_point['name'])

### Visualize and remap tiles

Usage notes:

* Proposed tiles are drawn in black. Automatically-generated tiles are drawn in blue.
* The selected proposed tile and its mapped automatically-generated tile are bolded.
* The `Proposed tile` tab can be used to visualize current mappings to automatically-generated tiles.
* The `Automatically-generated tile` tab is used for re-mapping a tile created by this script to one of your's.

In [11]:
%matplotlib widget
tiling_utils.interactive_remap(proposed_to_auto_map, proposed_tiles_info, auto_tiles_info, slide_data, figsize=(15, 15))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Output()